In [ ]:
def get_color(row, scalarMap):
    """Get the color to use based off the proportion of rides starting there.

    :param row: pandas dataframe row.
    :param scalarMap: Matplotlib scalarMappable object.
    :return: Hex color to use for area.
    """

    intensity = row['ride_proportion']

    colorVal = scalarMap.to_rgba(intensity)
    colorVal = (int(colorVal[0] * 255), int(colorVal[1] * 255),
                int(colorVal[2] * 255))

    intensity = "#{:02X}".format(colorVal[0]) + "{:02X}".format(colorVal[1]) + \
                "{:02X}".format(colorVal[2])

    return intensity
       

def interactive_map(mdp):
    """Interactive MDP map with node number, earn rate, drive time.
    
    :param mdp: mdp object with the needed data.
    :return fig: plotly fig plot, to plot use plot(fig) from plotly.offline.
    """
    
    neighborhoods = mdp.neighborhoods

    # Getting the centroid of all areas that were kept as nodes.
    centroids = {}

    for i in mdp.mapping.keys():
        latitude = neighborhoods[i][:, 0]
        longitude = neighborhoods[i][:, 1]

        centroid = (sum(latitude)/len(neighborhoods[i]),
                    sum(longitude)/len(neighborhoods[i]))
        centroids[i] = np.array(centroid)


    # For areas not kept as nodes, finding the area that was kept that was closest.
    area_pairs = {}

    for j in range(len(neighborhoods)):
        if j in mdp.mapping.keys():
            continue

        ref_polygon = Polygon(neighborhoods[j])
        point = ref_polygon.representative_point().xy

        lat = point[0][0]
        longitude = point[1][0]
        centroid = np.array([lat, longitude])

        dists = [(key, np.linalg.norm(centroids[key] - centroid)) for
                 key in centroids]

        new_area = sorted(dists, key=lambda x: x[1])[0][0]

        area_pairs[j] = new_area

    area_counts = pd.DataFrame(data=range(len(neighborhoods)), columns=['start_trip_area'])

    # Node start location by neighborhood index.
    area_counts['start_trip_area'] = area_counts['start_trip_area'].apply(lambda x: area_pairs[x] if x in area_pairs else x)

    # Converting neighborhood index to mdp node area.
    area_counts['start_trip_area'] = area_counts['start_trip_area'].apply(lambda x: mdp.mapping[x])

    area_counts['ride_proportion'] = area_counts['start_trip_area'].apply(lambda x: mdp.demand[x,0])

    # Creating the polygon coordinates for each of the areas.
    polygons = []
    centroids = []

    for i in neighborhoods:

        ref_polygon = Polygon(neighborhoods[i])
        point = ref_polygon.representative_point().xy

        lat = point[0][0]
        longitude = point[1][0]

        centroid = [lat, longitude]
        centroids.append(centroid)

        multi_poly = MultiPolygon([[coord[1], coord[0]] for coord in neighborhoods[i].tolist()])
        multi_poly['coordinates'] = [[multi_poly['coordinates']]]

        full_geo = {}
        full_geo['geometry'] = multi_poly
        full_geo['properties'] = {}
        full_geo['type'] = 'Feature'

        polygons.append(full_geo)

    centroids = np.vstack((centroids))
    lats = centroids[:,0]
    longs = centroids[:,1]

    area_counts['lat_centroids'] = lats    
    area_counts['long_centroids'] = longs   
    area_counts['coordinates'] = polygons

    # Getting earn rates as strings for the plot.
    earn_rate_avg = mdp.earn_rate_avg.tolist()
    for i in range(len(earn_rate_avg)):
        earn_rate_str = [str(j) + ' = ' + str(round(earn_rate_avg[i][j], 1)) 
                         if j % 5 != 0 or j == 0 else '<br>' + str(j) + ' = ' 
                         + str(round(earn_rate_avg[i][j], 2)) 
                         for j in range(len(earn_rate_avg[i]))]

        earn_rate_avg[i] = ', '.join(earn_rate_str)


    # Getting drive time as strings for the plot.
    drive_time_avg = mdp.drive_time_avg.tolist()
    for i in range(len(drive_time_avg)):
        drive_time_str = [str(j) + ' = ' + str(round(drive_time_avg[i][j], 1)) 
                          if j % 5 != 0 or j == 0 else '<br>' + str(j) + ' = ' 
                          + str(round(drive_time_avg[i][j], 2)) 
                          for j in range(len(drive_time_avg[i]))]

        drive_time_avg[i] = ', '.join(drive_time_str)

    area_counts['earn_rate'] = area_counts['start_trip_area'].apply(lambda x: earn_rate_avg[x])
    area_counts['drive_time'] = area_counts['start_trip_area'].apply(lambda x: drive_time_avg[x])

    # Column to hold color to use for the area on the neighborhood map.
    area_counts['color'] = pd.Series([None for row in xrange(len(area_counts))], index=area_counts.index)

    # Create the color map for the plot using the max and min values of ride proportion.
    cm = plt.cm.hot_r
    cNorm = colors.Normalize(vmin=min(area_counts['ride_proportion'].values),
                             vmax=max(area_counts['ride_proportion'].values))
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cm)
    scalarMap.get_clim()

    # Getting the color to use for area based off the ride proportion.
    area_counts['color'] = area_counts.apply(get_color, args=(scalarMap,), axis=1)  

    # Text to display on the plotly map.
    area_counts['text'] = '<br>' + 'Node: ' + area_counts['start_trip_area'].astype(str) + '<br><br>' \
                      + 'Earn Rates: ' +  area_counts['earn_rate'] + '<br><br>' \
                      + 'Drive Times: ' + area_counts['drive_time']

    mapbox_access_token = 'pk.eyJ1IjoiZmllenQiLCJhIjoiY2oxOGk5MXVqMDYyNjJ3b3ZiNHY1bTl0eSJ9.-6-DnmvlmyNtOQUsOhzdRg'

    # Creating layers of boundaries and colors for map layout.
    layers_ls = []
    for index, row in area_counts.iterrows():
        item_dict = dict(sourcetype = 'geojson', source = row['coordinates'],
                         type = 'fill', color = row['color'])
        layers_ls.append(item_dict)

    # Creating the full layout for the heatmap.
    layout = Layout(title = 'New York Taxi Data: Portion of Rides per Area',
                               height=1200, width=1200, autosize=False, hovermode='closest',
                               mapbox=dict(layers=layers_ls, accesstoken=mapbox_access_token,
                               bearing=0, center=dict(lat=40.704280,lon=-73.958805), pitch=0,
                               zoom=10.3,style='light'),)

    # Creating the color scale, list of lists containing ride proportion and color, low to high. 
    cscl_ = sorted(area_counts.color.unique(), reverse=True)
    cscl = []
    for col in cscl_:
        cscl.append([cNorm(min(area_counts.loc[area_counts['color'] == col]['ride_proportion'].unique())), col])


    # Adding the colorscale and all else for map.
    data = Data([Scattermapbox(lat = area_counts['lat_centroids'], 
                            lon = area_counts['long_centroids'], hoverinfo='lat+lon+text',
                            marker=Marker(cmax=max(area_counts['ride_proportion'].values),
                            cmin=min(area_counts['ride_proportion'].values), colorscale=cscl,
                            showscale = True, autocolorscale=False, size=0), text=area_counts['text'], 
                            mode = 'markers', textfont=dict(family='Courier New, monospace', size=18, color='#7f7f7f')),])

    fig = dict(data=data, layout=layout)
    
    return fig

In [ ]:
def node_visual(mdp, filename='policy.html'):
    """Plotting the states bounds in unique colors.

    :param mdp: mdp object containing the needed variables.
    :param filename: Path to save the file to.

    :return color_map: Dictionary from color to node index.
    """

    gmap = gmplot.GoogleMapPlotter(40.723031, -73.931419, 12)

    colors = ['blue','green','red','cyan','magenta','yellow','black','white','brown']

    color_map = {}

    idx = 0

    for key in mdp.mapping:
        if key == -1:
            color_map[mdp.mapping[key]] = 'blank'
            continue
        
        color_map[mdp.mapping[key]] = colors[idx]

        node = mdp.mapping[key]
        
        label = [k for k in mdp.num2state if mdp.num2state[k][0] == node]
        label = 'Node is ' + str(node) + ', States are ' + str(label).strip('[]')
        
        points = mdp.neighborhoods[key]
        latitude = points[:,0]
        longitude = points[:,1]
        
        centroid = (sum(latitude) / len(points), sum(longitude) / len(points))
        
        gmap.polygon(latitude, longitude, face_color=colors[idx], face_alpha=0.99)
        
        idx += 1
        
        gmap.draw(filename)

    return color_map